<a href="https://colab.research.google.com/github/elangbijak4/Riset-Smart-City/blob/main/Demo_AI_Driven_(Kasus_MLP)_untuk_EMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Simulasi dataset
def generate_data(n=1000):
    data = []
    for _ in range(n):
        speed = np.random.randint(0, 100)
        slope = np.random.choice(['flat', 'uphill', 'downhill'])
        battery = np.random.uniform(0, 1)

        # Simulasi aturan keputusan
        if slope == 'uphill' and battery < 0.3:
            action = 'engine'
        elif speed < 40 and battery > 0.5:
            action = 'battery'
        else:
            action = np.random.choice(['battery', 'engine'], p=[0.5, 0.5])

        data.append([speed, slope, battery, action])
    return pd.DataFrame(data, columns=['speed', 'slope', 'battery', 'action'])

df = generate_data()

# One-hot encoding untuk slope
df_encoded = pd.get_dummies(df, columns=['slope'])

# Encode label output (battery/engine) ke angka
label_enc = LabelEncoder()
df_encoded['action'] = label_enc.fit_transform(df_encoded['action'])  # battery=0, engine=1

# Split dataset
# Ensure X is explicitly converted to a float type NumPy array
X = df_encoded.drop(columns=['action']).values.astype(np.float32)
y = tf.keras.utils.to_categorical(df_encoded['action'].values, num_classes=2)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model MLP
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Training
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1)

# Evaluasi
loss, acc = model.evaluate(X_test, y_test)
print(f"\nTest Accuracy: {acc:.2f}")

# Prediksi input baru
def predict_action(speed, slope, battery):
    slope_dummies = {'slope_downhill': 0, 'slope_flat': 0, 'slope_uphill': 0}
    slope_dummies[f'slope_{slope}'] = 1
    # Ensure input_data is also explicitly converted to a float type
    input_data = np.array([[speed, battery] + list(slope_dummies.values())], dtype=np.float32)
    pred = model.predict(input_data)
    class_idx = np.argmax(pred)
    return label_enc.inverse_transform([class_idx])[0]

# Tes
print("Prediksi aksi untuk speed=50, slope='uphill', battery=0.2:",
      predict_action(50, 'uphill', 0.2))

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,530 (9.88 KB)

 Trainable params: 2,530 (9.88 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.4976 - loss: 2.1530 - val_accuracy: 0.4625 - val_loss: 0.8028
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5362 - loss: 0.7417 - val_accuracy: 0.5875 - val_loss: 0.7310
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5578 - loss: 0.7029 - val_accuracy: 0.5875 - val_loss: 0.7171
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5501 - loss: 0.7047 - val_accuracy: 0.6750 - val_loss: 0.6244
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6376 - loss: 0.6386 - val_accuracy: 0.6875 - val_loss: 0.6085
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6233 - loss: 0.6499 - val_accuracy: 0.6625 - val_loss: 0.6242
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6161 - loss: 0.6438 - val_accuracy: 0.6750 - val_loss: 0.6061
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6230 - loss: 0.6281 - val_accuracy: 0.6625 - val_loss: